In [1]:
import os 
import sys
import pandas as pd
import time
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json
import datetime
import sqlite3
import logging
%load_ext autoreload
%autoreload 2

In [2]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time().strftime('%H:%M:%S'))
print(current_dt)

2020-02-15 12:31:13.824815


In [3]:
link_homepage = 'https://meh.com'
page = requests.get(link_homepage)
print('Status Code', page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')

Status Code 200


## Save Raw Response

In [5]:
# os.chdir('..')
db_location = '../data/meh_scraper_qa.db'

In [6]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [7]:
stats = soup.find(class_='community-stats')

In [8]:
stats1 = str(stats)
stats2 = stats1.replace('\'','\'\'')

In [11]:
cursor.execute("""INSERT INTO raw_site_community_stats (date, time, raw_site) 
        VALUES ('{0}', '{1}', '{2}')""".format(current_date, current_time, stats2))

In [12]:
con.commit()

In [13]:
con.close()

## Parse Wanted Fields

In [14]:
parsed_nums = {}
parsed_nums['date'] = current_date
parsed_nums['time'] = current_time

### Page Views

In [15]:
page_views = stats.find('div', {'id':'pageviews'})

In [16]:
page_view_secondary = page_views.find(class_='secondary').get_text()
secondary_parts = page_view_secondary.rstrip().lstrip().split(', ')
for visit_method, method_num in zip(secondary_parts, range(1,3)):
    visit_parts = visit_method.split('% ')
    parsed_nums['Page Views ' + str(method_num)+' Name'] = visit_parts[1]
    parsed_nums['Page Views ' + str(method_num)+' Percent'] = int(visit_parts[0])

In [17]:
primary_class = page_views.find_all(class_='primary')
primary_numbs = [x.find('strong').get_text() for x in primary_class]
parsed_nums['# Visited'] = int(primary_numbs[0])
parsed_nums['Clicked Meh'] = int(primary_numbs[1])


In [18]:
parsed_nums

{'date': '2020-02-15',
 'time': '12:31:13',
 'Page Views 1 Name': 'on a phone',
 'Page Views 1 Percent': 51,
 'Page Views 2 Name': 'on a tablet.',
 'Page Views 2 Percent': 6,
 '# Visited': 40769,
 'Clicked Meh': 3016}

### Referrals

In [19]:
referrals = stats.find('div', {'id':'referrals'})

In [20]:
parsed_nums['Typed Meh Percent'] = int(referrals.find(class_='primary').find('strong').get_text()[:-1])

In [21]:
parsed_nums['Referrals Percent'] = int(referrals.find(class_='secondary').find('strong').get_text()[:-1])

In [22]:
referrers = referrals.find_all(class_='referrer')
for ref, ref_num in zip(referrers, range(1, len(referrers)+1)):
    parsed_nums['Referrer '+str(ref_num) + ' Percent'] = float(ref['data-percentage'])
    parsed_nums['Referrer '+str(ref_num) + ' Name'] = ref.get_text().lstrip().rstrip().split(' \n')[0]
    

### Order Counts

In [23]:
order_stats = stats.find('div', {'id': 'order-stats'})

In [24]:
order_nums = [re.findall('\d+', x.get_text())[0] for x in order_stats.find_all(class_='primary')]

In [25]:
parsed_nums['Items Sold'] = order_nums[0]
parsed_nums['Dollars Sold'] = order_nums[1]

## Export

In [26]:
export_series = pd.DataFrame(parsed_nums, index=[1])

In [27]:
export_series

,date,time,Page Views 1 Name,Page Views 1 Percent,Page Views 2 Name,Page Views 2 Percent,# Visited,Clicked Meh,Typed Meh Percent,Referrals Percent,...,Referrer 2 Percent,Referrer 2 Name,Referrer 3 Percent,Referrer 3 Name,Referrer 4 Percent,Referrer 4 Name,Referrer 5 Percent,Referrer 5 Name,Items Sold,Dollars Sold
1,2020-02-15,12:31:13,on a phone,51,on a tablet.,6,40769,3016,96,4,...,0.171555,dealnews.com,0.163385,facebook.com,0.117093,baidu.com,0.054462,morningsave.com,193,12071


In [28]:
con = sqlite3.connect(db_location)
cursor = con.cursor()
export_series.to_sql('selling_details', con, if_exists='append', index=False)

/home/pi/meh_scraper/meh_scraper_env/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [29]:
con.commit()
con.close()